<a href="https://colab.research.google.com/github/Ignacio-Ibarra/NLP-Disasters/blob/master/Eliminando_valores_contradictorios_en_target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Eliminando valores contradictorios de la variable `target`

Librerias


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
from wordcloud import WordCloud
import PIL
import itertools
from collections import defaultdict
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import string



%matplotlib inline

#para suprimir notacion cientifica en los outputs
pd.options.display.float_format='{:20,.2f}'.format

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Google Drive

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
link='https://drive.google.com/open?id=1Wuo42Fju9VOh2m5PV7q95TTUAIkrEwDD'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1Wuo42Fju9VOh2m5PV7q95TTUAIkrEwDD


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv')

Carga DF

In [44]:
#Loading 'train.csv' into DataFrame
train=pd.read_csv('train.csv')
train.head(10) #Let's have a first view of the DataFrame

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [0]:
print('El DataFrame cuenta con {} filas y {} columnas'.format(train.shape[0],train.shape[1]))

El DataFrame cuenta con 7613 filas y 5 columnas


In [54]:
train.groupby('text').agg({'target':['count','nunique']}).sort_values(by=('target','nunique'),ascending=False).head(15)

target        
                                                    count nunique
text                                                             
like for the music video I want some real actio...      2       2
Hellfire! We donÛªt even want to think about i...      2       2
The Prophet (peace be upon him) said 'Save your...      6       2
In #islam saving a person is equal in reward to...      2       2
To fight bioterrorism sir.                              4       2
Who is bringing the tornadoes and floods. Who i...      3       2
#foodscare #offers2go #NestleIndia slips into l...      3       2
#Allah describes piling up #wealth thinking it ...      3       2
He came to a land which was engulfed in tribal ...      6       2
RT NotExplained: The only known image of infamo...      2       2
Hellfire is surrounded by desires so be careful...      3       2
CLEARED:incident with injury:I-495  inner loop ...      3       2
Mmmmmm I'm burning.... I'm burning buildings I'...      2       2
wowo--=== 12000 Nigerian refugees repatriated f...      2       2
.POTUS #StrategicPatience is a strategy for #Ge...      4       2

Nos parece importante no tener 'target' que sean contradictorios si tenemos textos iguales. 

Parte del _cleanning_ incluye quitar las partes del texto que  sean palabras como _url_, _@mentions_, _#hashtags_

Para eso creamos la columna ['clean_text'], así podemos mantener las columna texto sin modificaciones. 

In [0]:
#Paso todo a minusculas en una columna nueva. 
train['clean_text']=train['text'].apply(lambda x: x.lower())

#Saco los url, mentions y hashtags de los textos
train['clean_text']=train['clean_text'].str.replace('http\S+|HTTP\S+|www.\S+|WWW.\S+|@\S+|#\S+', '', case=False)

In [0]:
#Agrupo por texto, calculo el 'nunique' de los targets y hago el transform para agregar columna al dataframe
train['target_nunique']=train.groupby('clean_text')['target'].transform('nunique')

In [0]:
#Cambio variable para no pisar 'train'
train2=train

#Me quedo sólo con los que tienen 'target_nunique'==1
train2=train[train.loc[:,'target_nunique']==1]

#Borro la columna 'target_nunique' en ambas variables. 
train.drop(columns='target_nunique',inplace=True)
train2.drop(columns='target_nunique',inplace=True)

In [57]:
print(train.shape)
print(train2.shape)

(7613, 6)
(7364, 6)


El dataset queda con 7364 filas.